In [22]:
import pyomo.environ as pyo

In [24]:

def build_toll_model():
    model = pyo.ConcreteModel()

    # Parameters
    model.D = pyo.Param(initialize=20)
    model.max_toll = pyo.Param(initialize=5)
    model.gamma = pyo.Param(['a', 'b', 'c', 'd'], initialize={'a': 10, 'b': 5, 'c': 8, 'd': 4})
    
    # Variables for tolls and flows
    model.toll_ab = pyo.Var(bounds=(0, model.max_toll))
    model.toll_cd = pyo.Var(bounds=(0, model.max_toll))
    model.flow_ab = pyo.Var(bounds=(0, 15))  # max flow for route ab
    model.flow_cd = pyo.Var(bounds=(0, 20))  # max flow for route cd
    
    # Objective: Maximize toll revenue
    model.revenue = pyo.Objective(expr=model.toll_ab * model.flow_ab + model.toll_cd * model.flow_cd, sense=pyo.maximize)

    # Constraint: Demand satisfaction
    model.demand = pyo.Constraint(expr=model.flow_ab + model.flow_cd == model.D)
    
    # KKT conditions based on lower-level optimization
    # Here we are assuming lower-level optimizes a cost function of toll and base costs
    # Dual variables for route choice conditions
    model.lambda_ab = pyo.Var(within=pyo.NonNegativeReals)
    model.lambda_cd = pyo.Var(within=pyo.NonNegativeReals)

    # Stationarity conditions for each route, integrating the base costs
    def stationarity_ab(model):
        return model.lambda_ab == model.gamma['a'] + model.gamma['b'] + model.toll_ab
    model.stationarity_ab = pyo.Constraint(rule=stationarity_ab)

    def stationarity_cd(model):
        return model.lambda_cd == model.gamma['c'] + model.gamma['d'] + model.toll_cd
    model.stationarity_cd = pyo.Constraint(rule=stationarity_cd)

    # Complementarity slackness conditions for flow not exceeding maximum capacity
    def comp_slack_ab(model):
        return (15 - model.flow_ab) * model.lambda_ab == 0
    model.comp_slack_ab = pyo.Constraint(rule=comp_slack_ab)

    def comp_slack_cd(model):
        return (20 - model.flow_cd) * model.lambda_cd == 0
    model.comp_slack_cd = pyo.Constraint(rule=comp_slack_cd)

    return model

# Create the model
model = build_toll_model()

# Solve the model using a solver that supports bilevel optimization
solver = pyo.SolverFactory('ipopt')
solver.solve(model)

# Output the results
print("Optimal tolls and flows with KKT conditions applied:")
print(f"Toll on route ab: {pyo.value(model.toll_ab)}, Flow on route ab: {pyo.value(model.flow_ab)}")
print(f"Toll on route cd: {pyo.value(model.toll_cd)}, Flow on route cd: {pyo.value(model.flow_cd)}")

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Ipopt 3.14.12\x3a Converged to a locally infeasible
      point. Problem may be infeasible.
Optimal tolls and flows with KKT conditions applied:
Toll on route ab: 2.3269321948732153e-05, Flow on route ab: 15.0
Toll on route cd: -9.997965266116669e-09, Flow on route cd: 4.999999999999999
